In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

import gc
import warnings
warnings.simplefilter("ignore")

### The dataset is loaded from a smaller size pickle file due to memory constraints

Thanks to colum2131 for his [kernel](https://www.kaggle.com/columbia2131/speed-up-reading-csv-to-pickle) that reduces the size of the dataset!


In [ ]:
# samples = 10000
train_df = pd.read_pickle('../input/ump-train-picklefile/train.pkl')#[:samples]

In [ ]:

train_df.head()

#Selecting the features to predict with
features = [f'f_{i}' for i in range(300)]

train_df.info()


In [ ]:
print("Unique ids: ",train_df["investment_id"].nunique(),"\n")

train_df["investment_id"].value_counts()

In [ ]:
train_df.isnull().sum().sum() #No null values

In [ ]:
investment_ids = train_df["investment_id"].unique()
num_ids = len(investment_ids)
train_size = 0.8

train_ids = investment_ids[:int(num_ids * train_size)]
valid_ids = investment_ids[int(num_ids * train_size):]

train = train_df[train_df["investment_id"].isin(train_ids)]
valid = train_df[train_df["investment_id"].isin(valid_ids)]

del train_df;gc.collect()

### Splitting data into train and validation

In [ ]:
train_X = train[["investment_id"]+features].values
valid_X = valid[["investment_id"]+features].values

train_y = train.target.values
valid_y = valid.target.values

### Scaling features and creating lgbm
Deleting old DataFrames to save memory

In [ ]:
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
valid_X = scaler.transform(valid_X)

lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=[0],free_raw_data=False)
lgb_eval = lgb.Dataset(valid_X,valid_y,categorical_feature=[0],free_raw_data=False)

del train_X; del valid_X;gc.collect()

### Training the model

In [ ]:
params = {
        "random_state": 420, 
        "verbosity": -2,
        "metrics": "rmse",
        "n_estimators":1500, 
        "learning_rate":0.1, 
        "subsample":0.8,
        "feature_fraction":0.6
}

model = lgb.train(
            params, lgb_train, valid_sets = lgb_eval,
            verbose_eval = False,
            num_boost_round = 1500,
            early_stopping_rounds = 100,
           
)

del lgb_train
del lgb_eval
gc.collect()

In [ ]:
import ubiquant
env = ubiquant.make_env()                   # initialize the environment
iter_test = env.iter_test()                 # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    preds = model.predict(test_df[["investment_id"] + features].values)
    sample_prediction_df["target"] = preds  # make your predictions here
    env.predict(sample_prediction_df)       # register your predictions